In [2]:
import torch
import numpy as np
import pandas as pd
import skimage.io
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
# import kornia as K
import cv2

import albumentations as A
# import tensorflow as tf
from sklearn.model_selection import train_test_split
# AUTOTUNE = tf.data.experimental.AUTOTUNE

In [3]:
# def additional_aug(p=1.0):
#     return A.Compose([
#         A.RandomRotate90(p=1.0)
#     ], p=p)

# img = cv2.cvtColor(cv2.imread("../input/hubmap-organ-segmentation/train_images/10044.tiff"), cv2.COLOR_BGR2RGB)
# mask = cv2.imread("../input/hacking-the-human-body-annotation-masks/train_binary_masks/10044.png", 0)
# data = {'image': img, 'mask': mask}
# transforms = additional_aug()
# transformed = transforms(**data)

# plt.figure(figsize=(12,14))
# plt.subplot(1,2,1)
# plt.imshow(transformed['image'])

# plt.subplot(1,2,2)
# plt.imshow(transformed['mask'])

In [4]:
df = pd.read_csv("/home/lakshita/somusan/hubmap_kaggle/hubmap_data/hubmap-organ-segmentation/train.csv")
lung_df = df[df["organ"]=='lung']
lung_df.head()
# df['organ'].value_counts() lung, spleen, largeintestine

,id,organ,data_source,img_height,img_width,pixel_size,tissue_thickness,rle,age,sex
3,10488,lung,HPA,3000,3000,0.4,4,3446519 15 3449517 17 3452514 20 3455510 24 34...,78.0,Male
13,11064,lung,HPA,3000,3000,0.4,4,2210040 68 2213039 70 2216036 74 2219032 79 22...,65.0,Male
18,11629,lung,HPA,3000,3000,0.4,4,4143554 1 4146554 2 4149554 3 4152555 3 415555...,43.0,Female
26,1220,lung,HPA,3000,3000,0.4,4,1429797 16 1432793 21 1435791 23 1438789 27 14...,59.0,Male
30,12452,lung,HPA,3000,3000,0.4,4,4648688 22 4651675 37 4654675 39 4657670 48 46...,78.0,Male


In [5]:
import imgaug as ia
import imgaug.augmenters as iaa
from tqdm import tqdm

!mkdir -p /home/lakshita/somusan/hubmap_kaggle/hubmap_data/lung_train/image
!mkdir -p /home/lakshita/somusan/hubmap_kaggle/hubmap_data/lung_train/mask

# pluge your augmentaions here
seq = iaa.Sequential([
    iaa.Rotate((-90, 180//2)),
    iaa.ScaleX((0.5, 2.3)),
    iaa.ShearY((-20, 20))
#     iaa.ShearX((-20, 20))
], random_order=True)

NUM_AUG_REPEAT = 2
for idx, i in tqdm(lung_df.iterrows()):
    for rep in range(NUM_AUG_REPEAT):
        img = cv2.cvtColor(cv2.imread("/home/lakshita/somusan/hubmap_kaggle/hubmap_data/hubmap-organ-segmentation/train_images/"+ str(i['id']) +".tiff"), cv2.COLOR_BGR2RGB)
        mask = cv2.imread("/home/lakshita/somusan/hubmap_kaggle/hubmap_data/mask_png/train_binary_masks/" + str(i['id']) + ".png", 0)

        h,w = mask.shape

        image, segmaps = seq(image=img, segmentation_maps=mask.reshape(1,h,w,1))
        segmaps = segmaps.reshape(h,w)
        mask_fname = "./lung_train/mask/" + str(i['id']) + "_" + str(rep) +".png"
        img_fname = "./lung_train/image/" + str(i['id'])+ "_" + str(rep) +".tiff"
        cv2.imwrite(mask_fname, segmaps)
        cv2.imwrite(img_fname, image)
    
# images_aug_i, segmaps_aug_i = seq(image=img, segmentation_maps=mask.reshape(1,3000,3000,1))
# # images_aug_i.shape

# plt.figure(figsize=(12,14))
# plt.subplot(1,2,1)
# plt.imshow(images_aug_i)

# plt.subplot(1,2,2)
# plt.imshow(segmaps_aug_i.reshape(3000,3000))

48it [00:37,  1.29it/s]


In [6]:
!ls /home/lakshita/somusan/hubmap_kaggle/hubmap_data/lung_train/mask

In [7]:
img = cv2.cvtColor(cv2.imread("/home/lakshita/somusan/hubmap_kaggle/hubmap_data/lung_train/image/16564_0.tiff"), cv2.COLOR_BGR2RGB)
mask = cv2.imread("/home/lakshita/somusan/hubmap_kaggle/hubmap_data/lung_train/mask/16564_0.png", 0)


plt.figure(figsize=(12,14))
plt.subplot(1,2,1)
plt.imshow(img)

plt.subplot(1,2,2)
plt.imshow(mask)


[ WARN:0@49.746] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/lakshita/somusan/hubmap_kaggle/hubmap_data/lung_train/image/16564_0.tiff'): can't open/read file: check file path/integrity


error: OpenCV(4.6.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
